In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
dir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees"
term_dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("old_terminus_helper.R"))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(latex2exp))
load(file.path(dir, "treeCons.RData"))
load(file.path(dir, "yAggConsNS.RData"))
load(file.path(dir, "treeCor.RData"))
load(file.path(dir, "yAggCorNS.RData"))
load(file.path(dir, "treeCons0.RData"))
load(file.path(dir, "yAggConsNS0.RData"))
load(file.path(dir, "treeMeanInf0.RData"))
load(file.path(dir, "yAggMeanInfNS0.RData"))
load(file.path(dir, "treeThr.RData"))
load(file.path(dir, "yAggThrNS.RData"))
load(file.path(dir, "yAggTermThrNS.RData"))
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))

In [2]:
dfScaledLFC <- data.frame(matrix(nrow=0, ncol=3))
colnames(dfScaledLFC) <- c("Type", "Value", "nCuts")

l <- length(treeCons$tip)
trees <- list(treeCons, treeCons0, treeMeanInf0, treeCor, treeThr)
names(trees) <- c("ConsInd", "Cons", "Mean", "AC", "ConsRedInd")
yAgg <- list(yAggConsNS, yAggConsNS0, yAggMeanInfNS0, yAggCorNS, yAggThrNS, yAggTermThrNS)
names(yAgg) <- c("ConsInd", "Cons", "Mean", "AC", "ConsRedInd", "Term")

dCutsSLFC <- vector(mode = "list", length=7)
names(dCutsSLFC) <- c(names(yAgg), "Txps")
for(i in seq_along(yAgg)) {
    
    if(i <= 5) {
        globArr <- rep(-100, l+trees[[i]]$Nnode)
        descL <- sapply(Descendants(trees[[i]], seq(l+trees[[i]]$Nnode)), length)
    }
        
    else {
        groupsClust <- parseClustFile(file.path(term_dir, "old", "1_1", "clusters.txt"), yAggTermThrNS, gsub=F)
        mtxps <- setdiff(seq(l), unlist(groupsClust))
        dCutsSLFC[[i]] <- c(mtxps, l+1:length(groupsClust))
        descL <- c(rep(1, length(mtxps)), sapply(groupsClust, length))
        globArr <- rep(-100, l+length(groupsClust))
    }
        
    infReps <- assays(yAgg[[i]])[grep("infRep", assayNames(yAgg[[i]]))]
    infReps <- abind::abind(as.list(infReps), along = 3)
    mSf <- 0
    for(j in seq(dim(infReps)[3])) {
            sf <- colSums(infReps[1:l,,j])
            mSf <- mean(sf) + mSf
            infReps[,,j] <- t(t(infReps[,,j])/sf)
    }
    mSf <- mSf/dim(infReps)[3]
    print(mSf)
    lfc <- getLog2FC(infReps, colData(yAgg[[i]])[["condition"]], pc = 5/mSf)
    obj <- abs(lfc)/mcols(yAgg[[i]])[["meanInfRV"]]
    if (i==6)         
        val <- sum(obj[dCutsSLFC[[i]]]*descL)

    else {
            val <- findMaxSum(trees[[i]], obj, l+1, descL)
            gc()
            dCutsSLFC[[i]] <- findCuts(trees[[i]], globArr, obj, l+1, descL)
            gc()
    }
    
    dfScaledLFC <- rbind(dfScaledLFC, data.frame(Type = names(yAgg)[i], Value = val,
           nCuts = length(dCutsSLFC[[i]])))

}
i <- 7
dCutsSLFC[[i]] <- 1:l
val <- sum(obj[dCutsSLFC[[i]]])
dfScaledLFC <- rbind(dfScaledLFC, data.frame(Type = "Txp", Value = val,
           nCuts = length(dCutsSLFC[[i]])))
print(dfScaledLFC)
save(dfScaledLFC, file=file.path(dir, "dfScaledLFC.RData"))
save(dCutsSLFC, file=file.path(dir, "dCutsSLFC.RData"))

[1] 33699064
[1] 33699064
[1] 33699064
[1] 33699064
[1] 33699064
[1] 33699064
        Type     Value  nCuts
1    ConsInd  963525.8  40796
2       Cons 1082263.7  32339
3       Mean 1086449.7  32607
4         AC  901094.1  85499
5 ConsRedInd  248532.3 122637
6       Term  189945.4 131323
7        Txp  167140.5 139535


In [2]:
load(file.path(dir, "dfInf.RData"))
load(file.path(dir, "dCutsDInf.RData"))
load(file.path(dir, "dCutsSLFC.RData"))

In [5]:
l <- length(treeCons$tip)
trees <- list(treeCons, treeCons0, treeMeanInf0, treeCor, treeThr)
names(trees) <- c("ConsFilt", "Cons", "Mean", "AC", "ConsFiltES")
yAgg <- list(yAggConsNS, yAggConsNS0, yAggMeanInfNS0, yAggCorNS, yAggThrNS)
names(yAgg) <- c("ConsFilt", "Cons", "Mean", "AC", "ConsFiltES", "Term")

ERROR: Error in names(yAgg) <- c("ConsFilt", "Cons", "Mean", "AC", "ConsFiltES", : 'names' attribute [6] must be the same length as the vector [5]


In [6]:
l <- length(treeCons$tip)
gammas <- c(0.1,0.5,1,5,10)
objsLFC <- vector(mode="list", length(yAgg))
names(objsLFC) <- names(yAgg)
objsDepth <- vector(mode="list", length(yAgg))
names(objsDepth) <- names(yAgg)
for(i in seq_along(yAgg)) {
    descL <- sapply(Descendants(trees[[i]],seq(l+trees[[i]]$Nnode)),length)
    objsDepth[[i]] <- (mcols(yAgg[[i]])[["meanInfRV"]] + 0.1*node.depth(trees[[i]],2))*descL
    infReps <- assays(yAgg[[i]])[grep("infRep", assayNames(yAgg[[i]]))]
    infReps <- abind::abind(as.list(infReps), along = 3)
    mSf <- 0
    for(j in seq(dim(infReps)[3])) {
            sf <- colSums(infReps[1:l,,j])
            mSf <- mean(sf) + mSf
            infReps[,,j] <- t(t(infReps[,,j])/sf)
    }
    mSf <- mSf/dim(infReps)[3]
    print(mSf)
    lfc <- getLog2FC(infReps, colData(yAgg[[i]])[["condition"]], pc = 5/mSf)
    objsLFC[[i]] <- descL*abs(lfc)/mcols(yAgg[[i]])[["meanInfRV"]]
}

In [7]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
i <- 6
infReps <- assays(yAggTermThrNS)[grep("infRep", assayNames(yAggTermThrNS))]
infReps <- abind::abind(as.list(infReps), along = 3)
term_dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
groupsClust <- parseClustFile(file.path(term_dir, "old", "1_1", "clusters.txt"), yAggTermThrNS, gsub=F)
mtxps <- setdiff(seq(l), unlist(groupsClust))

mSf <- 0
for(j in seq(dim(infReps)[3])) {
            sf <- colSums(infReps[1:l,,j])
            mSf <- mean(sf) + mSf
            infReps[,,j] <- t(t(infReps[,,j])/sf)
    }
mSf <- mSf/dim(infReps)[3]
descL <- c(rep(1, l), sapply(groupsClust, length))


lfc <- getLog2FC(infReps, colData(yAggTermThrNS)[["condition"]], pc = 5/mSf)
objsLFC[[i]] <- descL*abs(lfc)/mcols(yAggTermThrNS)[["meanInfRV"]]
names(objsLFC)[i] <- "Term"

#### Depth

In [8]:
for(i in seq(trees)){
    print(names(trees)[i])
    print(table(node.depth(trees[[i]],2)[dCutsDInf[[1]][[i]]])/length(dCutsDInf[[1]][[i]]))
}

[1] "ConsFilt"

           1            2            3            4            5            6 
8.134610e-01 5.758356e-02 3.842915e-02 3.122218e-02 2.231874e-02 1.489520e-02 
           7            8            9           10           11           12 
9.095941e-03 5.558630e-03 3.344804e-03 1.828813e-03 9.505017e-04 4.812667e-04 
          13           14           15           16           17           18 
3.007917e-04 2.767283e-04 1.082850e-04 2.406333e-05 2.406333e-05 1.203167e-05 
          19           21           29           30 
4.812667e-05 1.203167e-05 1.203167e-05 1.203167e-05 
[1] "Cons"

           1            2            3            4            5            6 
8.298766e-01 4.929017e-02 3.497036e-02 2.894215e-02 2.010152e-02 1.393884e-02 
           7            8            9           10           11           12 
8.907862e-03 5.748092e-03 3.339048e-03 2.005670e-03 1.187715e-03 6.386769e-04 
          13           14           15           16           17           1

In [34]:
names(dCutsDInf[[1]])
names(dCutsSLFC)
names(trees)

[1] "ConsInd"    "Cons"       "Mean"       "AC"         "ConsRedInd"
[6] "Txps"

[1] "ConsInd"    "Cons"       "Mean"       "AC"         "ConsRedInd"
[6] "Term"       "Txps"

[1] "ConsInd"    "Cons"       "Mean"       "AC"         "ConsRedInd"

In [9]:
i <- 1
dfDepth <- data.frame(depthObj = objsDepth[[i]][dCutsDInf[[1]][[i]]], type=names(objsDepth)[i])
for(i in seq(2,5)) {
    dfDepth <- rbind(dfDepth, data.frame(depthObj = objsDepth[[i]][dCutsDInf[[1]][[i]]], type=names(objsDepth)[i]))
}
size <- 20
dfDepth <- rbind(dfDepth, data.frame(depthObj = objsDepth[[i]][1:l], type="Txps"))
pBr <- ggplot(dfDepth, aes(x=depthObj, fill = type, colour = type)) + 
    geom_density(position = "dodge", alpha=0.1) +
    theme_bw() +
    scale_x_continuous(trans="log2") +
    labs(fill="Type", color="Type") +
    xlab("Value of Metric") +
    theme(axis.title = element_text(size = size),
    plot.title = element_text(size = size + 1, hjust=0.5),
    axis.text = element_text(size = size),
    legend.text = element_text(size = size),
    legend.title = element_text(size = size))

#### LFC

In [10]:
for(i in seq(trees)){
    print(names(trees)[i])
    print(table(node.depth(trees[[i]],2)[dCutsSLFC[[i]]])/length(dCutsSLFC[[i]]))
}
i <- 1
dfLFC <- data.frame(LFC = objsLFC[[i]][dCutsSLFC[[i]]], type=names(objsLFC)[i])
for(i in seq(2,6)) {
    dfLFC <- rbind(dfLFC, data.frame(LFC = objsLFC[[i]][dCutsSLFC[[i]]], type=names(objsLFC)[i]))
}
i <- 7
dfLFC <- rbind(dfLFC, data.frame(LFC = objsLFC[[i-1]][1:l], type="Txps"))
pBr <- ggplot(dfLFC, aes(x=log2(LFC), fill = type, colour = type)) + 
    geom_density(position = "dodge", alpha=0.1) +
    labs(fill="Type", color="Type") +
    xlab(TeX("$log_{2}(Value of Metric)$")) +
    theme_bw() +
    theme(axis.title = element_text(size = size),
    plot.title = element_text(size = size + 1, hjust=0.5),
    axis.text = element_text(size = size),
    legend.text = element_text(size = size),
    legend.title = element_text(size = size))

[1] "ConsFilt"

           1            2            3            4            5            6 
6.117266e-01 8.647907e-02 5.929503e-02 5.338759e-02 4.902441e-02 4.017551e-02 
           7            8            9           10           11           12 
2.794392e-02 2.169330e-02 1.524659e-02 1.073635e-02 7.843906e-03 5.294637e-03 
          13           14           15           16           17           18 
3.701343e-03 2.034513e-03 1.593293e-03 1.176586e-03 7.108540e-04 6.128052e-04 
          19           20           21           22           23           24 
3.676831e-04 3.676831e-04 1.960977e-04 1.715854e-04 2.451221e-05 7.353662e-05 
          25           26           29           41 
4.902441e-05 2.451221e-05 2.451221e-05 2.451221e-05 
[1] "Cons"

           1            2            3            4            5            6 
5.055506e-01 9.378769e-02 6.215406e-02 5.046538e-02 4.579610e-02 3.744705e-02 
           7            8            9           10           11           1

In [12]:
save(dfDepth, file=file.path(dir, "dfDe.RData"))
save(dfLFC, file=file.path(dir, "dfLL.RData"))